# Aula 10: Algoritmos em Grafos



## Objetivos de Aprendizagem

Ao final desta aula, você deverá ser capaz de:
1. Definir o problema de caminho mínimo de fonte única para pesos não-negativos.
2. Entender as pré-condições e limitações do Algoritmo de Dijkstra.
3. Implementar Dijkstra com complexidade $O((V + E)\log V)$.
4. Aplicar o Algoritmo de Bellman-Ford para grafos com pesos negativos e detectar ciclos.
5. Implementar Floyd-Warshall para caminhos mínimos todos-pares e analisar $O(n^3)$.
6. Aplicar Edmonds-Karp para cálculo de fluxo máximo com complexidade $O(VE^2)$.
7. Construir Árvores Geradoras Mínimas usando Prim e Kruskal com complexidades $O(E\log V)$ e $O(E\log E)$.

## 1. Dijkstra – Fonte Única (Recapitulando)
- Pressuposto: **pesos não-negativos**.
- Usa fila de prioridades para escolher vértice com menor distância provisória.


**Problema**: dado grafo $(G=(V,E))$ com pesos $(w(u,v)\ge0)$ e fonte $(s)$, encontrar $(d(s,v))$ mínimo.

**Pseudocódigo**:
```python
DIJKSTRA(adj, s):
    for u in V: dist[u] ← ∞
    dist[s] ← 0
    Q ← min-heap contendo (dist[u], u) para todos u
    while Q não vazio:
        (d, u) ← Q.pop()
        if d > dist[u]: continue        # entrada obsoleta
        for (v, w) in adj[u]:
            if dist[u] + w < dist[v]:
                dist[v] ← dist[u] + w
                Q.decrease_key(v, dist[v])
    return dist

In [ ]:
import heapq
def dijkstra(adj, source):
    dist = {u: float('inf') for u in adj}
    dist[source] = 0
    pq = [(0, source)]
    while pq:
        d, u = heapq.heappop(pq)
        if d > dist[u]: continue
        for v, w in adj[u]:
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return dist
# Exemplo
graph = {
    'A': [('B', 4), ('C', 1)],
    'B': [('C', 2), ('D', 5)],
    'C': [('D', 8), ('E', 10)],
    'D': [('E', 2)],
    'E': []
}
print('Dijkstra A->*:', dijkstra(graph, 'A'))

Complexidade: 
𝑂((∣𝑉∣+∣𝐸∣)log∣𝑉∣).

Pré-condição: todos os pesos devem ser ≥ 0.

## 2. Bellman-Ford – Fonte Única com Pesos Negativos
- Suporta **pesos negativos**, mas não suporta ciclos negativos.
- Relaxa todas as arestas até $|V|-1$ vezes e verifica ciclos.


````python
BELLMAN_FORD(adj, s):
    for u in V: dist[u] ← ∞
    dist[s] ← 0
    for i in 1..|V|-1:
        for (u, v, w) in todas as arestas:
            if dist[u] + w < dist[v]:
                dist[v] ← dist[u] + w
    # detectar ciclos negativos
    for (u, v, w) in todas as arestas:
        if dist[u] + w < dist[v]:
            erro "Ciclo negativo"
    return dist

````

In [ ]:
def bellman_ford(adj, source):
    dist = {u: float('inf') for u in adj}
    dist[source] = 0
    V = len(adj)
    for _ in range(V-1):
        for u in adj:
            for v, w in adj[u]:
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
    for u in adj:
        for v, w in adj[u]:
            if dist[u] + w < dist[v]:
                raise ValueError('Ciclo negativo detectado')
    return dist
# Exemplo com peso negativo
graph_neg = {
    'A': [('B', 1)],
    'B': [('C', -2)],
    'C': [('D', 1)],
    'D': []
}
print('Bellman-Ford A->*:', bellman_ford(graph_neg, 'A'))

## 3. Floyd-Warshall – Todos-Pares
- Calcula distâncias mínimas entre **todos os pares** de vértices.
- Complexidade: $O(n^3)$.


````python

FW(V, E):
    dist[i][j] ← ∞ para todo i,j
    dist[i][i] ← 0
    para cada aresta (i,j,w): dist[i][j] ← w
    for k in V:
      for i in V:
        for j in V:
          if dist[i][k] + dist[k][j] < dist[i][j]:
            dist[i][j] ← dist[i][k] + dist[k][j]
    return dist
````

In [ ]:
import pandas as pd
def floyd_warshall(vertices, edges):
    n = len(vertices)
    idx = {v:i for i,v in enumerate(vertices)}
    dist = [[float('inf')]*n for _ in range(n)]
    for i in range(n): dist[i][i] = 0
    for u, v, w in edges: dist[idx[u]][idx[v]] = w
    for k in range(n):
        for i in range(n):
            for j in range(n):
                nd = dist[i][k] + dist[k][j]
                if nd < dist[i][j]: dist[i][j] = nd
    return dist
vertices = ['A','B','C']
edges = [('A','B',5),('B','C',3),('A','C',10)]
matrix = floyd_warshall(vertices, edges)
print(pd.DataFrame(matrix, index=vertices, columns=vertices))

## 4. Fluxo Máximo – Edmonds-Karp
- Usa BFS para encontrar caminhos aumentantes.
- Complexidade: $O(VE^2)$.

O algoritmo Edmonds-Karp funciona usando a Busca em Largura (BFS) para encontrar um caminho com capacidade disponível da fonte até o coletor (chamado de caminho aumentado ) e, então, envia o máximo de fluxo possível por esse caminho.

O algoritmo Edmonds-Karp continua a encontrar novos caminhos para enviar mais fluxo até que o fluxo máximo seja atingido.

Animação - Edmonds-Karp https://www.w3schools.com/dsa/dsa_algo_graphs_edmondskarp.php


````python
EDMONDS_KARP(cap, s, t):
    flow[u][v] ← 0 para todo u,v
    max_flow ← 0
    while existe caminho p de s a t em grafo residual (BFS):
        path_flow ← min residual capacity em p
        para cada aresta (u→v) em p:
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
        max_flow += path_flow
    return max_flow
````

In [ ]:
from collections import deque
def edmonds_karp(capacity, source, sink):
    n = len(capacity)
    flow = [[0]*n for _ in range(n)]
    parent = [-1]*n
    max_flow = 0
    def bfs():
        nonlocal parent
        parent = [-1]*n
        parent[source] = source
        q = deque([source])
        while q:
            u = q.popleft()
            for v in range(n):
                if parent[v] < 0 and capacity[u][v] - flow[u][v] > 0:
                    parent[v] = u
                    if v == sink: return True
                    q.append(v)
        return False
    while bfs():
        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            path_flow = min(path_flow, capacity[u][v] - flow[u][v])
            v = u
        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = u
        max_flow += path_flow
    return max_flow
capacity = [[0,16,13,0,0,0],[0,0,10,12,0,0],[0,4,0,0,14,0],[0,0,9,0,0,20],[0,0,0,7,0,4],[0,0,0,0,0,0]]
print('Fluxo máximo:', edmonds_karp(capacity, 0, 5))

## 5. Árvores Geradoras Mínimas (MST - minimum spanning tree)

- **Prim**: usa heap, complexidade $O(E\log V)$.
- **Kruskal**: usa union-find, complexidade $O(E\log E)$.

Uma árvore geradora mínima (AGM), também conhecida como árvore de extensão mínima, é uma árvore que contém todos os vértices de um grafo conexo e não direcionado, e que tem o menor peso total de suas arestas entre todas as possíveis árvores geradoras. 

Em outras palavras, é a árvore que liga todos os nós do grafo com o menor custo total de arestas. 

Árvore Geradora Mínima - https://pt.wikipedia.org/wiki/%C3%81rvore_de_extens%C3%A3o_m%C3%ADnima#/media/Ficheiro:Minimum_spanning_tree.svg

Uma árvore geradora é chamada mínima se, dentre todas as árvores geradoras que existem no grafo, a soma dos pesos nas arestas dela é o menor possível.

````python
PRIM(G):
    escolha qualquer raiz r
    Q ← min-heap de arestas incidentes em r
    mark r
    MST ← ∅
    while |MST| < |V|-1:
        (u,v,w) ← Q.pop()
        if v não marcado:
            mark v
            MST.add((u,v,w))
            inserir em Q todas arestas de v para não marcados
    return MST
````

In [ ]:
import heapq
# Prim
def prim_mst(graph):
    start = next(iter(graph))
    visited = {start}
    edges = [(w, start, v) for v,w in graph[start]]
    heapq.heapify(edges)
    mst = []
    total = 0
    while edges and len(visited) < len(graph):
        w,u,v = heapq.heappop(edges)
        if v not in visited:
            visited.add(v)
            mst.append((u,v,w))
            total += w
            for nv,nw in graph[v]:
                if nv not in visited:
                    heapq.heappush(edges,(nw,v,nv))
    return total, mst
# Exemplo Prim
g = {'A':[('B',2),('C',3)],'B':[('A',2),('C',1),('D',4)],'C':[('A',3),('B',1),('D',5)],'D':[('B',4),('C',5)]}
print('MST Prim:', prim_mst(g))
# Kruskal
def kruskal_mst(vertices, edges):
    parent = {v:v for v in vertices}
    rank = {v:0 for v in vertices}
    def find(v):
        if parent[v]!=v: parent[v]=find(parent[v])
        return parent[v]
    def union(a,b):
        ra,rb = find(a), find(b)
        if ra==rb: return False
        if rank[ra]<rank[rb]: parent[ra]=rb
        else: parent[rb]=ra;
        if rank[ra]==rank[rb]: rank[ra]+=1
        return True
    mst=[]; total=0
    for u,v,w in sorted(edges, key=lambda x:x[2]):
        if union(u,v): mst.append((u,v,w)); total+=w
    return total, mst
vertices=['A','B','C','D']
edges=[('A','B',2),('A','C',3),('B','C',1),('B','D',4),('C','D',5)]
print('MST Kruskal:', kruskal_mst(vertices, edges))

#### Comparação rápida

| Algoritmo      | Tempo            | Espaço   | Restrições               |
| -------------- | ---------------- | -------- | ------------------------ |
| Dijkstra       | $O((V+E)\log V)$ | $O(V)$   | Pesos ≥ 0                |
| Bellman-Ford   | $O(VE)$          | $O(V)$   | Detecta ciclos negativos |
| Floyd-Warshall | $O(V^3)$         | $O(V^2)$ | —                        |
| Edmonds-Karp   | $O(VE^2)$        | $O(V^2)$ | —                        |
| Prim           | $O(E\log V)$     | $O(V)$   | —                        |
| Kruskal        | $O(E\log V)$     | $O(V)$   | —                        |


#### Dicas e Armadilhas
**Dijkstra:** nunca use com pesos negativos (use Bellman-Ford).

**Bellman-Ford:** cuidado com desempenho em grafos densos.

**Floyd-Warshall:** ótimo para grafos pequenos, todos-pares.

**Edmonds-Karp:** use em grafos de porte moderado, muita BFS residual.

**MST:** escolha Kruskal para conjuntos de arestas pré-ordenados, Prim para estruturas de lista de adjacência.